In [1]:
# Importing the necessary library's

import pandas as pd
import base64
import os
from requests import get, post
import json
import time
import numpy as np

Read in chart data, then isolate to only United States

In [2]:
# Reading the dataset and creating a dataframe
raw_data = pd.read_csv('charts.csv',
                       parse_dates=['date'])
raw_data = raw_data[raw_data['region'] == "United States"]
print(raw_data.isna().sum())
#stream numbers are missing only

title          0
rank           0
date           0
artist         0
url            0
region         0
chart          0
trend          0
streams    90873
dtype: int64


1. sort dataframe on date
2. drop any songs that show up on the same date, chart.. there are 57 of these and we are only keeping the first occurence
3. create new column 'song_num_days' which is a count of how many days a song is on either the top 50 playlist or the top 200 playlist
4. create new column 'num_days_cumulative' which is a cumulative count of how many days a song has been on each chart so far
5. create new column 'predict_days_stay'chart' which is how many more days the song will stay on that chart

In [3]:
raw_data.sort_values(by='date', inplace=True)
raw_data.drop_duplicates(subset=['chart','title', 'artist', 'date'], inplace=True, keep='first')

raw_data['predict_days_stay'] = raw_data.groupby(['chart','title','artist'])['date'].transform('nunique')
raw_data['num_days_cumulative'] = raw_data.groupby(['chart','title','artist'])['date'].cumcount()
raw_data['predict_days_stay'] = raw_data.apply(lambda row: row['predict_days_stay'] - row['num_days_cumulative'], axis = 1)

raw_data['Num_artists'] = raw_data.apply(lambda row: len(row.artist.split(',')), axis = 1)

raw_data

,title,rank,date,artist,url,region,chart,trend,streams,predict_days_stay,num_days_cumulative,Num_artists
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,429,0,1
8406,Wicked,160,2017-01-01,Future,https://open.spotify.com/track/6BbINUfGabVyiNF...,United States,top200,NEW_ENTRY,164503.0,1,0,1
8407,Black Barbies,161,2017-01-01,"Nicki Minaj, Mike WiLL Made-It",https://open.spotify.com/track/3y9cCbnBn0zjkJa...,United States,top200,MOVE_DOWN,163711.0,13,0,2
8408,Cut It (feat. Young Dolph),162,2017-01-01,O.T. Genasis,https://open.spotify.com/track/376KnY4TrgBITxj...,United States,top200,MOVE_UP,163667.0,1,0,1
8409,What They Want,163,2017-01-01,Russ,https://open.spotify.com/track/3pndPhlQWjuSoXh...,United States,top200,MOVE_DOWN,163498.0,338,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
22692001,The Spins,91,2021-12-31,Mac Miller,https://open.spotify.com/track/51pshtuYkgUQnt5...,United States,top200,MOVE_UP,274426.0,1,306,1
22692002,Dos Oruguitas,92,2021-12-31,Sebastian Yatra,https://open.spotify.com/track/5rohUzwEoRsUvAA...,United States,top200,MOVE_UP,273682.0,1,3,1
22692003,505,93,2021-12-31,Arctic Monkeys,https://open.spotify.com/track/58ge6dfP91o9oXM...,United States,top200,MOVE_DOWN,272088.0,1,328,1
22691990,Streets,80,2021-12-31,Doja Cat,https://open.spotify.com/track/60ynsPSSKe6O3sf...,United States,top200,MOVE_UP,287314.0,1,352,1


Next we will merge in the track data to the final dataset
I am merging based on track [title, artist]

In [4]:
tracks = pd.read_csv('charts_US_attributes.csv')

print("unique track,martist names: ",len(np.unique(raw_data[['title', 'artist']].values)))
print("track data we have: ",len(tracks))
print("track data we have: ",len(np.unique(tracks[['title', 'artist']].values)))
tracks

unique track,martist names:  17022
track data we have:  12683
track data we have:  17022


,title,rank,date,artist,url,region,chart,trend,streams,title_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,4Km5HrUvYTaSUfiSGPJeQR,...,-5.314,1,0.2440,0.061100,0.000000,0.1230,0.168,127.079,343150,4
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,343YBumqHu19cGoGARUTsd,...,-9.350,0,0.2870,0.105000,0.000000,0.1760,0.613,134.007,210937,4
2,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,5aAx2yezTd8zXrkmtKl66Z,...,-7.028,1,0.2820,0.165000,0.000003,0.1340,0.535,186.054,230453,4
3,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,7BKLCZ1jbUBVqRi2FVlTVw,...,-5.599,1,0.0338,0.414000,0.000000,0.1110,0.661,95.010,244960,4
4,Black Beatles,5,2017-01-01,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0,6fujklziTHa8uoM5OQSfIo,...,-6.163,1,0.0649,0.142000,0.000000,0.1280,0.355,145.926,291893,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,Butterflies,28,2021-07-19,Fase Yoda,https://open.spotify.com/track/6jDOs0wGlRn4jCg...,United States,viral50,NEW_ENTRY,NaN,6jDOs0wGlRn4jCgJ4Z7HZv,...,-9.299,1,0.1140,0.222000,0.000000,0.2670,0.332,130.026,118154,4
12679,Butterflies,36,2021-07-19,"MAX, Ali Gatie",https://open.spotify.com/track/7eQHxigpuDJjCG5...,United States,viral50,NEW_ENTRY,NaN,7eQHxigpuDJjCG50JyzU8v,...,-4.041,1,0.0408,0.385000,0.000000,0.0887,0.420,88.084,191250,4
12680,Colt 45 (Country Remix),37,2021-07-19,"Cooper Alan, Rvshvd",https://open.spotify.com/track/0kgRRHa5CMeCciI...,United States,viral50,NEW_ENTRY,NaN,0kgRRHa5CMeCciIgYT67re,...,-3.299,0,0.2030,0.293000,0.000000,0.1420,0.845,184.021,203804,4
12681,MMM MMM (feat. ATL Jacob),45,2021-07-19,Kali,https://open.spotify.com/track/7GLqR9ToJLb0PV3...,United States,viral50,NEW_ENTRY,NaN,7GLqR9ToJLb0PV3XyNAWNm,...,-8.690,1,0.3460,0.134000,0.000000,0.1130,0.187,86.533,137341,4


In [38]:
#print(raw_data.columns)
#print(tracks.columns)
track_columns = [ 'title', 
                 'artist', 
                 'danceability', 
                 'energy', 
                 'key', 
                 'loudness',
                 'mode', 
                 'speechiness', 
                 'acousticness', 
                 'instrumentalness', 
                 'liveness',
                 'valence', 
                 'tempo', 
                 'duration_ms', 
                 'time_signature']

tracks = tracks[track_columns]
final_data = pd.merge(raw_data, tracks, 
                      left_on=['title', 'artist'], 
                      right_on= ['title', 'artist'],
                      how='left')
print("raw:   ",  len(raw_data))
print("final: " , len(final_data))  
print("diff:  " , len(raw_data) - len(final_data))    
print(final_data.columns)
final_data.sort_values(by='date', inplace=True)
final_data.head()


raw:    455010
final:  455010
diff:   0
Index(['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend',
       'streams', 'predict_days_stay', 'num_days_cumulative', 'Num_artists',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature'],
      dtype='object')


,title,rank,date,artist,url,region,chart,trend,streams,predict_days_stay,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,429,...,-5.314,1,0.2440,0.0611,0.000000,0.123,0.168,127.079,343150,4
157,You Can Call Me Al,125,2017-01-01,Paul Simon,https://open.spotify.com/track/0qxYx4F3vm1AOnf...,United States,top200,NEW_ENTRY,183669.0,1,...,-8.124,1,0.0534,0.1820,0.006470,0.077,0.820,128.433,280000,4
158,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0,455,...,-9.350,0,0.2870,0.1050,0.000000,0.176,0.613,134.007,210937,4
159,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0,502,...,-7.028,1,0.2820,0.1650,0.000003,0.134,0.535,186.054,230453,4
160,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0,538,...,-5.599,1,0.0338,0.4140,0.000000,0.111,0.661,95.010,244960,4


next we will merge the artists data, queries from spotify, into the main dataset

In [46]:
artists_columns = [ 
                    'artist_id',
                    'artist_followers',
                    'genres',
                    'artist_popularity',
                    'artist_name',
                    'type',
                    'album_names',
                    'release_dates',
                    'artist_total_tracks'
                  ]
artists = pd.read_csv('artist_data.csv')
artists.rename(columns={'id': 'artist_id', 
                        'followers': 'artist_followers', 
                        'popularity': 'artist_popularity', 
                        'name': 'artist', 
                        'total_tracks':'artist_total_tracks'}, inplace=True )
artists['artist'].apply(lambda x: x.upper())


print(artists.columns)




Index(['artist_id', 'artist_followers', 'genres', 'artist_popularity',
       'artist', 'type', 'album_names', 'release_dates',
       'artist_total_tracks'],
      dtype='object')


In [44]:

print("total needed: ", len(final_data))
singleartists = final_data[final_data['Num_artists'] == 1]
singleartists['artist'].apply(lambda x: x.upper())

print("single needed: ", len(singleartists))
multipleartists = final_data[final_data['Num_artists'] > 1]
print("multiple needed: ", len(multipleartists))
print('artist data: ', len(artists))

final_single = pd.merge(singleartists, artists, 
                        on='artist', 
                        how='inner')
print("single found: ", len(final_single))

final_single

total needed:  455010
single needed:  382081
multiple needed:  72929
artist data:  4742
single found:  443774


,title,rank,date,artist,url,region,chart,trend,streams,predict_days_stay,...,time_signature,artist_id,artist_followers,genres,artist_popularity,artist_name,type,album_names,release_dates,artist_total_tracks
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0,429,...,4,6oMuImdp5ZcFhWP0ESe6mG,13504155,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",76,Migos,artist,"['Culture III (Deluxe)', 'Culture III', 'Cultu...","['2021-06-11', '2021-06-11', '2018-01-26', '20...",243
1,Bad and Boujee (feat. Lil Uzi Vert),27,2017-01-02,Migos,https://open.spotify.com/track/1FvU97lrWOG2NRx...,United States,viral50,NEW_ENTRY,NaN,15,...,4,6oMuImdp5ZcFhWP0ESe6mG,13504155,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",76,Migos,artist,"['Culture III (Deluxe)', 'Culture III', 'Cultu...","['2021-06-11', '2021-06-11', '2018-01-26', '20...",243
2,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-02,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1161746.0,428,...,4,6oMuImdp5ZcFhWP0ESe6mG,13504155,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",76,Migos,artist,"['Culture III (Deluxe)', 'Culture III', 'Cultu...","['2021-06-11', '2021-06-11', '2018-01-26', '20...",243
3,Bad and Boujee (feat. Lil Uzi Vert),24,2017-01-03,Migos,https://open.spotify.com/track/1FvU97lrWOG2NRx...,United States,viral50,MOVE_UP,NaN,14,...,4,6oMuImdp5ZcFhWP0ESe6mG,13504155,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",76,Migos,artist,"['Culture III (Deluxe)', 'Culture III', 'Cultu...","['2021-06-11', '2021-06-11', '2018-01-26', '20...",243
4,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-03,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1284891.0,427,...,4,6oMuImdp5ZcFhWP0ESe6mG,13504155,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",76,Migos,artist,"['Culture III (Deluxe)', 'Culture III', 'Cultu...","['2021-06-11', '2021-06-11', '2018-01-26', '20...",243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443769,ELEVEN,12,2021-12-31,IVE,https://open.spotify.com/track/7n2FZQsaLb7ZRfR...,United States,viral50,MOVE_DOWN,NaN,1,...,4,6RHTUrRF63xao58xh9FXYJ,2985898,['k-pop girl group'],76,IVE,artist,"[""I've IVE""]",['2023-04-10'],11
443770,Habang Buhay,20,2021-12-29,Zack Tabudlo,https://open.spotify.com/track/63bmIgH9sS6sX5S...,United States,viral50,NEW_ENTRY,NaN,3,...,4,67IN4cLJ7798gUapyZlmac,3445136,"['acoustic opm', 'opm', 'p-pop']",70,Zack Tabudlo,artist,"['ZACK: FOR ALL', 'Episode']","['2023-02-14', '2021-10-15']",24
443771,Habang Buhay,21,2021-12-30,Zack Tabudlo,https://open.spotify.com/track/63bmIgH9sS6sX5S...,United States,viral50,MOVE_DOWN,NaN,2,...,4,67IN4cLJ7798gUapyZlmac,3445136,"['acoustic opm', 'opm', 'p-pop']",70,Zack Tabudlo,artist,"['ZACK: FOR ALL', 'Episode']","['2023-02-14', '2021-10-15']",24
443772,Habang Buhay,21,2021-12-31,Zack Tabudlo,https://open.spotify.com/track/63bmIgH9sS6sX5S...,United States,viral50,SAME_POSITION,NaN,1,...,4,67IN4cLJ7798gUapyZlmac,3445136,"['acoustic opm', 'opm', 'p-pop']",70,Zack Tabudlo,artist,"['ZACK: FOR ALL', 'Episode']","['2023-02-14', '2021-10-15']",24
